# Create an interface between two materials with no strain matching

Interface created from two slabs with provided terminations. The interface is created without strain matching, so the film is wrapped into the substrate slab cell according to matrix ...????

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials
2. Set notebook parameters
2. Create slab configurations, view possible terminations and select the termination pair for the interface
3. Create the interface with no strain matching based on the selected configurations
4. Visualize the interface
5. Pass the interface to the outside runtime


## 1. Prepare Environment
### 1.1. Set notebook parameters

In [12]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 
# Enable scaling of the film slab to match the substrate lattice
ENABLE_FILM_SCALING = False


### 1.2. Install Packages
Note: the step executes only when using Pyodide. For other environments (e.g. Colab, JupyterLab), the packages should be installed via `pip install .` (see [README](../../README.ipynb)).

In [13]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb", "../../config.yml")

### 1.3. Get input materials and assign `substrate` and `film`
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [14]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
substrate = materials[5]
film = materials[1]

Data from 0-Ni has been read successfully.
Data from 1-Graphene has been read successfully.
Data from 2-WS2 has been read successfully.
Data from 3-BN has been read successfully.
Data from 4-Te2Mo has been read successfully.
Data from 5-HfO2 has been read successfully.
Data from Hf12 C2 O24, Interface has been read successfully.


### 1.4. Preview Substrate and Film

In [15]:
from utils.visualize import visualize_materials as visualize
visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

GridBox(children=(VBox(children=(Label(value='Hf4O8 - Material', layout=Layout(align_self='center', height='30…

## 2. Create Slabs and select termination pair

### 2.1. Create Substrate and Layer Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.
Define the substrate slab cell that will be used as a base for the interface and the film slab cell that will be placed on top of the substrate slab.

In [16]:
from mat3ra.made.tools.build.slab import SlabConfiguration

film_slab_configuration = SlabConfiguration(
    bulk=film,
    # the Miller indices of the interfacial plane of the layer
    miller_indices=(0, 0, 1),
    thickness=1,
    vacuum=1,
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

substrate_slab_configuration = SlabConfiguration(
    bulk=substrate,
    # the Miller indices of the interfacial plane of the substrate
    miller_indices=(1,1,1),
    thickness=3,
    vacuum=3,
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

film_slab_terminations = film_slab_configuration.terminations
substrate_slab_terminations = substrate_slab_configuration.terminations

print("Film terminations:")
for idx, term in enumerate(film_slab_terminations):
    print(f"{idx}: {term}")
print("Substrate terminations:")
for idx, term in enumerate(substrate_slab_terminations):
    print(f"{idx}: {term}")

Film terminations:
0: C_P6/mmm_2
Substrate terminations:
0: Hf_P-1_2
1: O2_P1_4
2: O2_P1_3
3: O2_P-1_2
4: O2_P2/m_1


### 2.2. Visualize slabs for all possible terminations

In [17]:
visualize([{"material":film_slab_configuration.get_material(termination=termination), "title":f"Termination: {termination}"} for termination in film_slab_terminations], repetitions=[3, 3, 1], rotation="-90x")

visualize([{"material":substrate_slab_configuration.get_material(termination=termination), "title":f"Termination: {termination}"} for termination in substrate_slab_terminations], repetitions=[3, 3, 1], rotation="-90x")


GridBox(children=(VBox(children=(Label(value='C2 - Termination: C_P6/mmm_2', layout=Layout(align_self='center'…

GridBox(children=(VBox(children=(Label(value='Hf4O8Hf4O8Hf4O8 - Termination: Hf_P-1_2', layout=Layout(align_se…

### 2.3. Select termination pair for the interface

In [18]:
from itertools import product
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

# Set the termination pair indices
substrate_termination_index = 0
film_termination_index = 0

termination_pair = (film_slab_terminations[film_termination_index], substrate_slab_terminations[substrate_termination_index])
termination_pairs = list(product(film_slab_terminations, substrate_slab_terminations))
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination_pair = await ui_prompt_select_array_element_by_index_pyodide(termination_pairs, element_name="film/substrate termination pair")
    else:
        termination_pair = ui_prompt_select_array_element_by_index(termination_pairs, element_name="film/substrate termination pair")

## 3. Create an interface
### 3.1. Initialize the Interface Configuration
We use no strain matching for the interface construction. Substrate slab cell is used as a base and the film slab is wrapped into it with optionally scaling atomic coordinates (preserve coordinates in crystal/alat units).

In [19]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration

interface_configuration = InterfaceConfiguration(substrate_slab_configuration, film_slab_configuration, termination_pair, distance_z=3.0, vacuum=5.0)

interface = interface_configuration.get_material()

### 3.2. Scale the film slab to match the substrate lattice (optional)

In [20]:
from mat3ra.made.tools.build.interface import InterfaceConfigurationStrainMatcher
if ENABLE_FILM_SCALING:
    film_scaling_strain_matcher = InterfaceConfigurationStrainMatcher(interface_configuration, strain_matching_parameters=None)
    interface = film_scaling_strain_matcher.get_interfaces()[0]

### 6.2. Visualize the interface


In [21]:
visualize([interface], repetitions=[3, 3, 1], rotation="0x")
visualize([interface], repetitions=[3, 3, 1], rotation="-90x")

<class 'mat3ra.made.material.Material'>


GridBox(children=(VBox(children=(Label(value='Hf4O8Hf4O8Hf4O8C2 - Material', layout=Layout(align_self='center'…

GridBox(children=(VBox(children=(Label(value='Hf4O8Hf4O8Hf4O8C2 - Material', layout=Layout(align_self='center'…

### 6.3. Pass data to the outside runtime
Enrich the interface names with the strain values and pass them to the application runtime.

In [22]:
from utils.jupyterlite import set_data

if "Interface" not in interface.name:
    interface.name = f'{interface.name}, Interface'
set_data("materials", [interface.to_json()])

Data for materials written to uploads/Hf12 C2 O24, Interface.json
